# Calcualte performance rankings from cleaned results

In [20]:
# read cleaned results
import pandas as pd
from pathlib import Path
agg_df_with_default = pd.read_csv(Path("./cleaned_results/tuned_aggregated_results_with_default.csv"))
agg_df = pd.read_csv(Path("./cleaned_results/tuned_aggregated_results.csv"))

fold_df_with_default = pd.read_csv(Path("./cleaned_results/tuned_fold_results_with_default.csv"))
fold_df = pd.read_csv(Path("./cleaned_results/tuned_fold_results.csv"))

# make sure output folder exists
output_folder = Path("./performance_rankings")
output_folder.mkdir(exist_ok=True)  

In [21]:
def get_rank_table(df, metric):
    """
    NOTE: the df needs to have the following columns defined:
    - {metric}_rank_mean
    - normalized_{metric}__test_mean
    """

    overall_ranks = df.groupby("alg_name").agg(
        {
            f"{metric}_rank_mean": ["min", "max", "mean", "count"],
            f"normalized_{metric}__test_mean": "mean",
        }
    ).reset_index().sort_values([(f"{metric}_rank_mean", "mean")])

    # format min/max rank columns to be ints

    overall_ranks.loc[:, "count"] = overall_ranks.loc[:, (f"{metric}_rank_mean", "count")].astype(int)
    overall_ranks.drop(columns=(f"{metric}_rank_mean", "count"), inplace=True)

    # overall_ranks.loc[:, "alg_name"] = overall_ranks.loc[:, "alg_name"].apply(lambda x: "\rot{" + x + "}")
    overall_ranks.loc[:, (f"{metric}_rank_mean", "min")] = overall_ranks.loc[:, (f"{metric}_rank_mean", "min")].astype(int)
    overall_ranks.loc[:, (f"{metric}_rank_mean", "max")] = overall_ranks.loc[:, (f"{metric}_rank_mean", "max")].astype(int)

    overall_ranks.loc[:, (f"{metric}_rank_mean", "mean")] = overall_ranks.loc[:, (f"{metric}_rank_mean", "mean")].round(2)
    
    overall_ranks.loc[:, (f"normalized_{metric}__test_mean", "mean")] = overall_ranks.loc[:,(f"normalized_{metric}__test_mean", "mean")].round(2)

    final_table = overall_ranks.set_index("alg_name")

    return final_table


### Save rank tables to file

In [22]:
# save rank tables to csv and latex
from analysis_utils import ALG_DISPLAY_NAMES

metric_list = [
    "Accuracy",
    "F1",
    "Log Loss",
]

# best, worst, and average performance for each alg, over all datasets
for metric in metric_list:

    agg_df_with_default

    # first with default hparams as its own alg
    final_table = get_rank_table(agg_df_with_default, metric)

    # save to csv, latex
    final_table.to_csv(output_folder / f"{metric}_rank_with_default.csv", index=True)
    final_table.to_latex(output_folder / f"{metric}_rank_with_default.tex", index=True, escape=False)

    # now without default hparams as its own alg
    final_table = get_rank_table(agg_df, metric)

    # save to csv, latex
    final_table.to_csv(output_folder / f"{metric}_rank.csv", index=True)
    final_table.to_latex(output_folder / f"{metric}_rank.tex", index=True, escape=False)